In [0]:
import duckdb
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

In [0]:
con = duckdb.connect("db_duckdb/balancacomercial.duckdb")
con.execute(""" ATTACH 'db_duckdb/cnpj.duckdb' AS cnpj (READ_ONLY)""")
con.execute(""" SHOW ALL TABLES""").df()

In [0]:
con.execute(""" SELECT DISTINCT cnpj_basico, cnpj_ordem, cnpj_dv FROM cnpj.estabelecimentos """).df().shape[0]

10524625

-- ANÁLISE 1: POTENCIAL EXPORTADOR POR ESTADO (UF)

-- Pergunta: A quantidade de empresas ativas em um estado reflete o volume exportado?

In [0]:
con.execute(
"""
    SELECT 
        exp.SG_UF_NCM AS uf,
        SUM(exp.VL_FOB) AS total_exportado_usd,
        SUM(exp.KG_LIQUIDO) AS peso_liquido_kg
    FROM fct_exp_2021 exp
    GROUP BY exp.SG_UF_NCM
""").df().head()

In [0]:
con.execute(
"""
SELECT 
    est.uf,
    COUNT(*) AS qtd_empresas_ativas
FROM cnpj.estabelecimentos est
WHERE est.situacao_cadastral = '02' -- Filtra apenas empresas ATIVAS
GROUP BY est.uf

"""
).df().head()

In [0]:
df = con.execute(
"""
WITH exportacao_estado AS (
    SELECT 
        exp.SG_UF_NCM AS uf,
        SUM(exp.VL_FOB) AS total_exportado_usd,
        SUM(exp.KG_LIQUIDO) AS peso_liquido_kg
    FROM fct_exp_2021 exp
    GROUP BY exp.SG_UF_NCM
),
empresas_ativas_estado AS (
    SELECT 
        est.uf,
        COUNT(*) AS qtd_empresas_ativas
    FROM cnpj.estabelecimentos est
    WHERE est.situacao_cadastral = '02' -- Filtra apenas empresas ATIVAS
    GROUP BY est.uf
)
SELECT 
    ex.uf AS UF,
    e.qtd_empresas_ativas AS Qtd_Empresas_Ativas,
    CAST(ex.total_exportado_usd AS DECIMAL(18,2)) AS Total_Exportado_USD,
    CAST((ex.total_exportado_usd / e.qtd_empresas_ativas) AS DECIMAL(18,2)) AS USD_por_Empresa_Ativa
FROM exportacao_estado ex
JOIN empresas_ativas_estado e ON ex.uf = e.uf
ORDER BY USD_por_Empresa_Ativa DESC; """

).df()

In [0]:
df.head()

In [0]:
df["Total_Exportado_Bi"] = df["Total_Exportado_USD"] / 1000000000

In [0]:
df.head()

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Configuração de Estilo
sns.set_theme(style="whitegrid")
plt.figure(figsize=(18, 12))

# ---------------------------------------------------------
# VISUALIZAÇÃO 1: SCATTER PLOT (MATRIZ DE ESTRUTURA)
# ---------------------------------------------------------
plt.subplot(2, 1, 1) # Posição superior

# Scatter plot: X=Empresas, Y=Exportação, Tamanho=KPI
scatter = sns.scatterplot(
    data=df, 
    x='Qtd_Empresas_Ativas', 
    y='Total_Exportado_Bi', 
    size='USD_por_Empresa_Ativa', 
    sizes=(50, 1000), 
    hue='USD_por_Empresa_Ativa',
    palette='viridis', 
    alpha=0.7,
    legend=False
)

# Adicionando rótulos nos pontos
for i in range(df.shape[0]):
    # Destacar apenas os principais para não poluir
    if df.USD_por_Empresa_Ativa[i] > 100000 or df.UF[i] in ['SP', 'RJ', 'DF']:
        plt.text(
            df.Qtd_Empresas_Ativas[i]+10000, 
            df.Total_Exportado_Bi[i], 
            df.UF[i], 
            fontweight='bold', 
            size=10
        )

plt.title('Matriz de Estrutura Econômica: Volume x Densidade Empresarial\n(O tamanho da bolha indica a eficiência exportadora)', fontsize=14)
plt.xlabel('Quantidade de Empresas Ativas (Base CNPJ)', fontsize=12)
plt.ylabel('Total Exportado 2021 (Bilhões USD)', fontsize=12)

# Desenhando quadrantes teóricos (linhas médias)
plt.axhline(y=df['Total_Exportado_Bi'].mean(), color='r', linestyle='--', alpha=0.3)
plt.axvline(x=df['Qtd_Empresas_Ativas'].mean(), color='r', linestyle='--', alpha=0.3)
plt.text(df['Qtd_Empresas_Ativas'].max()*0.8, df['Total_Exportado_Bi'].max()*0.1, "Quadrante: Alta Diversidade\n(Muitas empresas, foco interno)", fontsize=10, color='gray')
plt.text(50000, df['Total_Exportado_Bi'].max()*0.9, "Quadrante: Alta Concentração\n(Poucas empresas, muita exportação)", fontsize=10, color='gray')


# ---------------------------------------------------------
# VISUALIZAÇÃO 2: RANKING DE KPI (QUEM É MAIS EFICIENTE?)
# ---------------------------------------------------------
plt.subplot(2, 1, 2) # Posição inferior

# Ordenar dados
df_sorted = df.sort_values('USD_por_Empresa_Ativa', ascending=False).head(10)

# Barplot
barplot = sns.barplot(
    data=df_sorted, 
    x='USD_por_Empresa_Ativa', 
    y='UF', 
    palette='magma'
)

# Adicionar valores nas barras
for index, value in enumerate(df_sorted['USD_por_Empresa_Ativa']):
    plt.text(value + 5000, index, f"US$ {value:,.0f}", va='center', fontsize=10)

plt.title('Top 10 Estados por Eficiência Exportadora (USD Exportado por CNPJ Ativo)', fontsize=14)
plt.xlabel('Valor Médio Exportado por Empresa (USD)', fontsize=12)
plt.ylabel('Unidade Federativa', fontsize=12)

plt.tight_layout()
plt.show()

- PA e MT: poucas empresas e muita exportação (commodite)
- SP exporta mais que todo mundo mas tem um numero elevado de empresas